<a href="https://colab.research.google.com/github/mo-urad0/Social-Network-Lab/blob/main/Improved%20Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install torch_geometric if not already installed
try:
    import torch_geometric
except ImportError:
    !pip install torch_geometric

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid


# 1. Load the dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class ImprovedGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # IMPROVEMENT 1: Increase hidden dimension from 4 to 16
        hidden_channels = 16

        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Layer 1
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        # IMPROVEMENT 2: Add Dropout to prevent overfitting
        # standard dropout rate for Cora is 0.5
        x = F.dropout(x, p=0.5, training=self.training)

        # Layer 2
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedGCN().to(device)
data = data.to(device)

# IMPROVEMENT 3: Add Weight Decay (L2 Regularization)
# 5e-4 is the standard weight decay for GCN on Cora
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# IMPROVEMENT 4: Increase Epochs to 200
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # Optional: Print progress
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# Evaluate
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'----------------------------')
print(f'Optimized GCN Test Accuracy: {acc:.2f}')

Epoch 0, Loss: 1.9422
Epoch 20, Loss: 0.2132
Epoch 40, Loss: 0.0622
Epoch 60, Loss: 0.0383
Epoch 80, Loss: 0.0391
Epoch 100, Loss: 0.0328
Epoch 120, Loss: 0.0436
Epoch 140, Loss: 0.0336
Epoch 160, Loss: 0.0474
Epoch 180, Loss: 0.0242
----------------------------
Optimized GCN Test Accuracy: 0.81


In [8]:
pip install torch